Importing the Data

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
prices=pd.read_csv('/Users/sanjogniroula/Downloads/airbnb_price.csv')
print(prices.head())

In [5]:
xls=pd.ExcelFile('/Users/sanjogniroula/Downloads/airbnb_room_type.xlsx')
room_types=xls.parse(0)        

In [6]:
print(room_types.head())

   listing_id                                description        room_type
0        2595                      Skylit Midtown Castle  Entire home/apt
1        3831            Cozy Entire Floor of Brownstone  Entire home/apt
2        5099  Large Cozy 1 BR Apartment In Midtown East  Entire home/apt
3        5178            Large Furnished Room Near B'way     private room
4        5238         Cute & Cozy Lower East Side 1 bdrm  Entire home/apt


In [8]:
reviews=pd.read_csv('/Users/sanjogniroula/Downloads/airbnb_last_review.tsv',sep='\t')
print(reviews.head())

   listing_id    host_name   last_review
0        2595     Jennifer   May 21 2019
1        3831  LisaRoxanne  July 05 2019
2        5099        Chris  June 22 2019
3        5178     Shunichi  June 24 2019
4        5238          Ben  June 09 2019


Cleaning the price column

In [10]:
prices['price']=prices['price'].str.replace(r'\D','',regex=True)
print(prices)

       listing_id price                 nbhood_full
0            2595   225          Manhattan, Midtown
1            3831    89      Brooklyn, Clinton Hill
2            5099   200      Manhattan, Murray Hill
3            5178    79   Manhattan, Hell's Kitchen
4            5238   150        Manhattan, Chinatown
...           ...   ...                         ...
25204    36425863   129  Manhattan, Upper East Side
25205    36427429    45            Queens, Flushing
25206    36438336   235  Staten Island, Great Kills
25207    36442252   100           Bronx, Mott Haven
25208    36455809    30          Brooklyn, Bushwick

[25209 rows x 3 columns]


In [11]:
prices['price']=pd.to_numeric(prices['price'])
print(prices['price'])

0        225
1         89
2        200
3         79
4        150
        ... 
25204    129
25205     45
25206    235
25207    100
25208     30
Name: price, Length: 25209, dtype: int64


Calculating average price

In [37]:
free_listings=prices['price']==0
prices=prices.loc[~free_listings]
avg_price=prices['price'].mean()
print("The average price per night for an Airbnb listing in NYC is ${0:.2f}.".format(avg_price))

The average price per night for an Airbnb listing in NYC is $141.82.


Comparing costs to the private rental market

In [40]:
prices['price_per_month']=prices['price']*365/12
average_price_per_month=round(prices['price_per_month'].mean(),2)
print('airbnb monthly costs are ${}, while in the private market you would pay {})'.format(average_price_per_month,"$3100.00"))

airbnb monthly costs are $4313.61, while in the private market you would pay $3100.00)


Cleaning the room type column

In [41]:
room_types["room_type"]=room_types["room_type"].str.lower()
room_types["room_type"]=room_types["room_type"].astype("category")
room_frequencies=room_types["room_type"].value_counts()
print(room_frequencies)

entire home/apt    13266
private room       11356
shared room          587
Name: room_type, dtype: int64


What timeframe are we working with?

In [43]:
reviews["last_review"]=pd.to_datetime(reviews["last_review"])
first_reviewed=reviews["last_review"].dt.date.min()
last_reviewed=reviews["last_review"].dt.date.max()
print("The latest Airbnb review is {}, the earliest review is {}".format(first_reviewed,last_reviewed))

The latest Airbnb review is 2019-01-01, the earliest review is 2019-07-09


Joining the DataFrames

In [44]:
rooms_and_prices=prices.merge(room_types,how='outer',on='listing_id')
airbnb_merged=rooms_and_prices.merge(reviews,how='outer',on='listing_id')
airbnb_merged.dropna(inplace=True)
print("There are {} duplicates in the DataFrame.".format(airbnb_merged.duplicated().sum()))

There are 0 duplicates in the DataFrame.


Analyzing listing prices by NYC borough

In [45]:
airbnb_merged["borough"]=airbnb_merged["nbhood_full"].str.partition(', ')[0]
boroughs=airbnb_merged.groupby("borough")["price"].agg(["sum","mean","median","count"])
boroughs=boroughs.round(2).sort_values("mean",ascending=False)
print(boroughs)

                     sum    mean  median  count
borough                                        
Manhattan      1898417.0  184.04   149.0  10315
Brooklyn       1275250.0  122.02    95.0  10451
Queens          320715.0   92.83    70.0   3455
Staten Island    22974.0   86.04    71.0    267
Bronx            55156.0   79.25    65.0    696


Analyzing listing prices by NYC borough

In [46]:
label_names=["Budget","Average","Expensive","Extravagant"]
ranges=[0,69,175,350,np.inf]
airbnb_merged["price_range"]=pd.cut(airbnb_merged["price"],bins=ranges,labels=label_names)
prices_by_borough=airbnb_merged.groupby(['borough','price_range'])['price_range'].count()
print(prices_by_borough)

borough        price_range
Bronx          Budget          381
               Average         285
               Expensive        25
               Extravagant       5
Brooklyn       Budget         3194
               Average        5532
               Expensive      1466
               Extravagant     259
Manhattan      Budget         1148
               Average        5285
               Expensive      3072
               Extravagant     810
Queens         Budget         1631
               Average        1505
               Expensive       291
               Extravagant      28
Staten Island  Budget          124
               Average         123
               Expensive        20
               Extravagant       0
Name: price_range, dtype: int64
